# How to use the process area

Follow along this step-by-step guide to learn about the [`ProcessArea`](https://geospaitial-lab.github.io/aviary/api_reference/process_area).

To avoid any issues, run the cells in order and don't skip any cells.<br />
If something seems off, just restart the runtime and run the cells again.

# Install aviary

Install aviary in the current runtime using pip.

In [ ]:
! pip install -q geospaitial-lab-aviary

# Import aviary and verify the installation

In [ ]:
import aviary

print(aviary.__version__)

# Create a process area

A process area specifies the area of interest by a set of coordinates of the bottom left corner of each tile.

By default, a new instance of the [`ProcessArea`](https://geospaitial-lab.github.io/aviary/api_reference/process_area) has no coordinates.<br />
You can access the coordinates of the process area with the `coordinates` attribute,
which is a numpy array of shape (n, 2) and data type int32.

In [ ]:
process_area = aviary.ProcessArea()

print(process_area.coordinates)

If you already have the coordinates, you can pass them to the initializer of the [`ProcessArea`](https://geospaitial-lab.github.io/aviary/api_reference/process_area).

In [ ]:
import numpy as np

coordinates = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area = aviary.ProcessArea(coordinates=coordinates)

print(process_area.coordinates)

We can visualize the process area with [folium](https://python-visualization.github.io/folium/latest) for a better understanding.

Install folium in the current runtime using pip.

In [ ]:
! pip install -q folium

We define a function `visualize_process_area`, so that we can reuse it in the next steps.

In [ ]:
import folium
import geopandas as gpd


def visualize_process_area(
    process_area: aviary.ProcessArea,
    tile_size: int,
    zoom_start: int = 16,
) -> folium.Map:
    # Convert the process area to a geodataframe
    gdf = process_area.to_gdf(
        tile_size=tile_size,
        epsg_code=25832,
    )

    # Compute the centroid of the process area
    centroid = gpd.GeoDataFrame(
        geometry=[gdf.union_all().centroid],
        crs=gdf.crs,
    )

    # Convert the centroid to EPSG:4326 (folium requires EPSG:4326)
    centroid_epsg_4326 = centroid.to_crs(epsg=4326)

    # Compute the location of the folium map
    location_epsg_4326 = [
        centroid_epsg_4326.geometry.y.mean(),
        centroid_epsg_4326.geometry.x.mean(),
    ]

    # Convert the process area to EPSG:4326 (folium requires EPSG:4326)
    gdf_epsg_4326 = gdf.to_crs(epsg=4326)

    # Create a folium map
    folium_map = folium.Map(
        location=location_epsg_4326,
        zoom_start=zoom_start,
        tiles='OpenStreetMap',
    )

    # Define the style of the process area
    style_function = lambda feature: {
        'fillOpacity': .2,
        'color': 'black',
        'weight': 2,
    }

    # Add the process area to the folium map
    folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

    return folium_map

Now we can visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

folium_map

You can set the coordinates of an already created process area with the `coordinates` attribute.

In [ ]:
coordinates = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
    ],
    dtype=np.int32,
)
process_area.coordinates = coordinates

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

folium_map

A process area is an iterable object, so it supports indexing, length and iteration.

You can access the coordinates of the process area with the index operator.

In [ ]:
coordinates_1 = process_area[0]
coordinates_2 = process_area[1]

print(coordinates_1)
print(coordinates_2)

You can slice the process area to create a new process area of a subset of the coordinates with the index operator
and the colon operator.

In [ ]:
sliced_process_area = process_area[:-1]

print(sliced_process_area.coordinates)

A process area has a length, which is equal to the number of coordinates, i.e. the number of tiles.

In [ ]:
print(len(process_area))

You can iterate over the coordinates of the process area.

In [ ]:
for coordinates in process_area:
    print(coordinates)

## Create a process area from a bounding box

You can create a process area from a bounding box with the
[`from_bounding_box`](https://geospaitial-lab.github.io/aviary/api_reference/process_area/#aviary.ProcessArea.from_bounding_box) class method.

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)
process_area = aviary.ProcessArea.from_bounding_box(
    bounding_box=bounding_box,
    tile_size=128,
    quantize=False,
)

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

# Convert the bounding box to a geodataframe
gdf = bounding_box.to_gdf(epsg_code=25832)

# Convert the bounding box to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the bounding box to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

You can set the tile size of the process area with the `tile_size` parameter.<br />
If the bounding box is not divisible by the tile size, the tiles will extend beyond the bounding box.

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)
process_area = aviary.ProcessArea.from_bounding_box(
    bounding_box=bounding_box,
    tile_size=96,
    quantize=False,
)

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=96,
)

# Convert the bounding box to a geodataframe
gdf = bounding_box.to_gdf(epsg_code=25832)

# Convert the bounding box to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the bounding box to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

You can quantize the process area with the `quantize` parameter.<br />
If the coordinates are not divisible by the tile size, the coordinates will be quantized to the tile size.<br />
This might be useful when you want to ensure matching tiles for different process areas.

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)
process_area = aviary.ProcessArea.from_bounding_box(
    bounding_box=bounding_box,
    tile_size=128,
    quantize=True,
)

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

# Convert the bounding box to a geodataframe
gdf = bounding_box.to_gdf(epsg_code=25832)

# Convert the bounding box to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the bounding box to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

## Create a process area from a geodataframe

You can create a process area from a geodataframe with the
[`from_gdf`](https://geospaitial-lab.github.io/aviary/api_reference/process_area/#aviary.ProcessArea.from_gdf) class method.

In [ ]:
from shapely.geometry import box

gdf = gpd.GeoDataFrame(
    geometry=[box(363084, 5715326, 363340, 5715582)],
    crs='EPSG:25832',
)
process_area = aviary.ProcessArea.from_gdf(
    gdf=gdf,
    tile_size=128,
    quantize=False,
)

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

# Convert the geodataframe to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the geodataframe (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the geodataframe to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

The geodataframe may contain multiple polygons, e.g. the northern districts of Gelsenkirchen.

In [ ]:
url = (
    'https://raw.githubusercontent.com/geospaitial-lab/aviary/main'
    '/docs/how_to_guides/api/data/districts.geojson'
)
gdf = gpd.read_file(url)
process_area = aviary.ProcessArea.from_gdf(
    gdf=gdf,
    tile_size=256,
    quantize=True,
)

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=256,
    zoom_start=12,
)

# Convert the districts to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the districts (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the districts to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

## Create a process area from a json string

You can create a process area from a json string with the
[`from_json`](https://geospaitial-lab.github.io/aviary/api_reference/process_area/#aviary.ProcessArea.from_json) class method.

In [ ]:
json_string = (
    '[[363084, 5715326], '
    '[363212, 5715326], '
    '[363084, 5715454], '
    '[363212, 5715454]]'
)
process_area = aviary.ProcessArea.from_json(json_string=json_string)

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

folium_map

# Add, subtract or intersect process areas

You can add two process areas with the `+` operator.<br />
If the process areas overlap, the resulting process area will contain the union of the two process areas.

In [ ]:
coordinates_1 = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area_1 = aviary.ProcessArea(coordinates=coordinates_1)

coordinates_2 = np.array(
    [
        [363212, 5715454],
        [363340, 5715454],
        [363212, 5715582],
        [363340, 5715582],
    ],
    dtype=np.int32,
)
process_area_2 = aviary.ProcessArea(coordinates=coordinates_2)

print(process_area_1.coordinates)
print(process_area_2.coordinates)

In [ ]:
process_area = process_area_1 + process_area_2

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

# Convert the first process area to a geodataframe
gdf_1 = process_area_1.to_gdf(
    tile_size=128,
    epsg_code=25832,
)

# Convert the first process area to EPSG:4326 (folium requires EPSG:4326)
gdf_1_epsg_4326 = gdf_1.to_crs(epsg=4326)

# Define the style of the first process area (red)
style_function_1 = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the first process area to the folium map
folium.GeoJson(gdf_1_epsg_4326, style_function=style_function_1).add_to(folium_map)

# Convert the second process area to a geodataframe
gdf_2 = process_area_2.to_gdf(
    tile_size=128,
    epsg_code=25832,
)

# Convert the second process area to EPSG:4326 (folium requires EPSG:4326)
gdf_2_epsg_4326 = gdf_2.to_crs(epsg=4326)

# Define the style of the second process area (blue)
style_function_2 = lambda feature: {
    'fillOpacity': 0.,
    'color': '#1982C4',
    'weight': 2,
}

# Add the second process area to the folium map
folium.GeoJson(gdf_2_epsg_4326, style_function=style_function_2).add_to(folium_map)

folium_map

You can subtract two process areas with the `-` operator.

In [ ]:
coordinates_1 = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area_1 = aviary.ProcessArea(coordinates=coordinates_1)

coordinates_2 = np.array(
    [
        [363212, 5715454],
        [363340, 5715454],
        [363212, 5715582],
        [363340, 5715582],
    ],
    dtype=np.int32,
)
process_area_2 = aviary.ProcessArea(coordinates=coordinates_2)

print(process_area_1.coordinates)
print(process_area_2.coordinates)

In [ ]:
process_area = process_area_1 - process_area_2

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

# Convert the first process area to a geodataframe
gdf_1 = process_area_1.to_gdf(
    tile_size=128,
    epsg_code=25832,
)

# Convert the first process area to EPSG:4326 (folium requires EPSG:4326)
gdf_1_epsg_4326 = gdf_1.to_crs(epsg=4326)

# Define the style of the first process area (red)
style_function_1 = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the first process area to the folium map
folium.GeoJson(gdf_1_epsg_4326, style_function=style_function_1).add_to(folium_map)

# Convert the second process area to a geodataframe
gdf_2 = process_area_2.to_gdf(
    tile_size=128,
    epsg_code=25832,
)

# Convert the second process area to EPSG:4326 (folium requires EPSG:4326)
gdf_2_epsg_4326 = gdf_2.to_crs(epsg=4326)

# Define the style of the second process area (blue)
style_function_2 = lambda feature: {
    'fillOpacity': 0.,
    'color': '#1982C4',
    'weight': 2,
}

# Add the second process area to the folium map
folium.GeoJson(gdf_2_epsg_4326, style_function=style_function_2).add_to(folium_map)

folium_map

You can intersect two process areas with the `&` operator.

In [ ]:
coordinates_1 = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area_1 = aviary.ProcessArea(coordinates=coordinates_1)

coordinates_2 = np.array(
    [
        [363212, 5715454],
        [363340, 5715454],
        [363212, 5715582],
        [363340, 5715582],
    ],
    dtype=np.int32,
)
process_area_2 = aviary.ProcessArea(coordinates=coordinates_2)

print(process_area_1.coordinates)
print(process_area_2.coordinates)

In [ ]:
process_area = process_area_1 & process_area_2

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

# Convert the first process area to a geodataframe
gdf_1 = process_area_1.to_gdf(
    tile_size=128,
    epsg_code=25832,
)

# Convert the first process area to EPSG:4326 (folium requires EPSG:4326)
gdf_1_epsg_4326 = gdf_1.to_crs(epsg=4326)

# Define the style of the first process area (red)
style_function_1 = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the first process area to the folium map
folium.GeoJson(gdf_1_epsg_4326, style_function=style_function_1).add_to(folium_map)

# Convert the second process area to a geodataframe
gdf_2 = process_area_2.to_gdf(
    tile_size=128,
    epsg_code=25832,
)

# Convert the second process area to EPSG:4326 (folium requires EPSG:4326)
gdf_2_epsg_4326 = gdf_2.to_crs(epsg=4326)

# Define the style of the second process area (blue)
style_function_2 = lambda feature: {
    'fillOpacity': 0.,
    'color': '#1982C4',
    'weight': 2,
}

# Add the second process area to the folium map
folium.GeoJson(gdf_2_epsg_4326, style_function=style_function_2).add_to(folium_map)

folium_map

# Append coordinates to the process area

You can append coordinates to the process area with the
[`append`](https://geospaitial-lab.github.io/aviary/api_reference/process_area/#aviary.ProcessArea.append) method.

In [ ]:
coordinates = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area = aviary.ProcessArea(coordinates=coordinates)

print(process_area.coordinates)

In [ ]:
process_area = process_area.append((363340, 5715582))

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

folium_map

If you want to append coordinates that already exist, the process area will not change.

In [ ]:
process_area = process_area.append((363340, 5715582))

print(process_area.coordinates)

# Chunk the process area

You can chunk the process area into multiple process areas with the
[`chunk`](https://geospaitial-lab.github.io/aviary/api_reference/process_area/#aviary.ProcessArea.chunk) method.<br />
This might be useful when you want to run multiple pipelines in distributed environments.

In [ ]:
coordinates = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area = aviary.ProcessArea(coordinates=coordinates)

print(process_area.coordinates)

In [ ]:
process_areas = process_area.chunk(num_chunks=2)

for process_area in process_areas:
    print(process_area.coordinates)

# Filter the process area

You can filter the process area with the
[`filter`](https://geospaitial-lab.github.io/aviary/api_reference/process_area/#aviary.ProcessArea.filter) method.<br />
This method applies a
[`CoordinatesFilter`](https://geospaitial-lab.github.io/aviary/api_reference/geodata/coordinates_filter/coordinates_filter)
to the coordinates of the process area.

In this example, we will filter the process area based on geospatial data with the
[`GeospatialFilter`](https://geospaitial-lab.github.io/aviary/api_reference/geodata/coordinates_filter/geospatial_filter).<br />
You can remove coordinates of tiles that are within the polygons in the geodataframe with the difference mode.

In [ ]:
coordinates = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area = aviary.ProcessArea(coordinates=coordinates)

gdf = gpd.GeoDataFrame(
    geometry=[box(363212, 5715454, 363468, 5715710)],
    crs='EPSG:25832',
)

print(process_area.coordinates)
print(gdf)

In [ ]:
from aviary.geodata import GeospatialFilter

geospatial_filter = GeospatialFilter(
    tile_size=128,
    gdf=gdf,
    mode=aviary.GeospatialFilterMode.DIFFERENCE,
)
process_area = process_area.filter(coordinates_filter=geospatial_filter)

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

# Convert the geodataframe to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the geodataframe (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the geodataframe to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

You can remove coordinates of tiles that don't intersect with the polygons in the geodataframe with the intersection mode.

In [ ]:
coordinates = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area = aviary.ProcessArea(coordinates=coordinates)

gdf = gpd.GeoDataFrame(
    geometry=[box(363212, 5715454, 363468, 5715710)],
    crs='EPSG:25832',
)

print(process_area.coordinates)
print(gdf)

In [ ]:
geospatial_filter = GeospatialFilter(
    tile_size=128,
    gdf=gdf,
    mode=aviary.GeospatialFilterMode.INTERSECTION,
)
process_area = process_area.filter(coordinates_filter=geospatial_filter)

print(process_area.coordinates)

Visualize the process area.

In [ ]:
folium_map = visualize_process_area(
    process_area=process_area,
    tile_size=128,
)

# Convert the geodataframe to EPSG:4326 (folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the geodataframe (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#FF595E',
    'weight': 2,
}

# Add the geodataframe to the folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

# Convert the process area to a geodataframe

You can convert the process area to a geodataframe with the
[`to_gdf`](https://geospaitial-lab.github.io/aviary/api_reference/process_area/#aviary.ProcessArea.to_gdf) method.

In [ ]:
coordinates = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area = aviary.ProcessArea(coordinates=coordinates)

print(process_area.coordinates)

In [ ]:
gdf = process_area.to_gdf(
    tile_size=128,
    epsg_code=25832,
)

print(gdf)

# Convert the process area to a json string

You can convert the process area to a json string with the
[`to_json`](https://geospaitial-lab.github.io/aviary/api_reference/process_area/#aviary.ProcessArea.to_json) method.

In [ ]:
coordinates = np.array(
    [
        [363084, 5715326],
        [363212, 5715326],
        [363084, 5715454],
        [363212, 5715454],
    ],
    dtype=np.int32,
)
process_area = aviary.ProcessArea(coordinates=coordinates)

print(process_area.coordinates)

In [ ]:
json_string = process_area.to_json()

print(json_string)